In [19]:
!pip install statsmodels

Defaulting to user installation because normal site-packages is not writeable


In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import statsmodels.api as sm
import statsmodels.formula.api as smf


In [3]:
# web_page_data.csv 파일 읽기
# 페이지, 머무른시간 사이에 유의미한 차이가 있는지 검정
# 페이지별로 2세트의 머무른 시간 데이터를 만들어서 검정
df_web_page = pd.read_csv('data-files/web_page_data.csv')
df_web_page[:3]

,Page,Time
0,Page A,0.21
1,Page B,2.53
2,Page A,0.35


In [4]:
# 페이지별 머무른 시간 평균
df_web_page.groupby(by=['Page'])['Time'].mean()

Page
Page A    1.263333
Page B    1.620000
Name: Time, dtype: float64

In [7]:
df_web_page.shape

(36, 2)

In [ ]:
# Page A에 해당하는 Time 데이터 선택
page_a_mask = df_web_page['Page'] == 'Page A'
time_in_page_a = df_web_page.loc[page_a_mask, 'Time']

# Page B에 해당하는 Time 데이터 선택
page_b_mask = df_web_page['Page'] == 'Page B'
time_in_page_b = df_web_page.loc[page_b_mask, 'Time']

time_in_page_a.shape, time_in_page_b.shape

((21,), (15,))

In [ ]:
# 수치형(연속형) 2표본 평균 비교 검정 ( 두 집단의 평균 비교 )
stats.ttest_ind(time_in_page_a, time_in_page_b)

TtestResult(statistic=np.float64(-1.1237042154424823), pvalue=np.float64(0.26901024363926024), df=np.float64(34.0))

In [12]:
# 요일별로 팁에 차이가 있는지 검정
df_tips = sns.load_dataset('tips')
df_tips[:5]

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [ ]:
# 요일별 팁 평균 비교
df_tips.groupby(by=['day'])['tip'].mean()

C:\Users\human\AppData\Local\Temp\ipykernel_13308\1092338430.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_tips.groupby(by=['day'])['tip'].mean()


day
Thur    2.771452
Fri     2.734737
Sat     2.993103
Sun     3.255132
Name: tip, dtype: float64

In [22]:
# 요일별로 tip 데이터 준비해서 stats.ttest_ind로 검정
# 문제 : stats.ttest_ind는 두 집단(샘플)에 대해서만 검정
# 해결 : 세 집단 이상의 평균 비교는 분산분석 (ANOVA)

model = smf.ols("tip~day", data=df_tips) # tip~day : tip = f(day)
model = model.fit()
sm.stats.anova_lm(model) # 회귀모델을 기반으로 분산분석 (세 집단의 평균 비교) 실행

,df,sum_sq,mean_sq,F,PR(>F)
day,3.0,9.525873,3.175291,1.672355,0.173589
Residual,240.0,455.686604,1.898694,NaN,NaN
